In [1]:
import arlpy.uwapm as pm
import numpy as np

x = np.linspace(-7500,10000, 1080)
z = np.linspace(-15, 3100,  720)

tx_freq  = 5000
tx_depth = 500
tx_angle = np.linspace(-180, 180 , 10)
tx_level = 10*np.sin(2*np.pi*16*tx_angle/360)+10*np.sin(2*np.pi*8*tx_angle/360)

surfaceWaveHeight = 3 # m
surfaceWaveLength = 7 # m
top_range         = np.array([-13000, 1000, 5000, 10000])
top_interface     = surfaceWaveHeight*np.sin(2*np.pi/surfaceWaveLength*top_range)

ssp_range = 0
ssp_depth = np.array([100, 1000, 5000])
ssp       = np.array([1200, 1600, 1400])

bot_range     = np.array([-10000, -500, 5000, 7000, 10000])
bot_interface = np.array([2000, 3100, 3000, 2000, 1500])

env = pm.make_env2d(
                
        pad_inputData   = True, 
        name            = 'Example',

        # BELLHOP mode and attn
        mode            = 'incoherent', 
        volume_attn     = 'Thorp',
        
        # ALL: Receiver positoins
        rx_range        = x,                                                   # m
        rx_depth        = z,                                                   # m
        
        # OALIB: Top boundary condition
        top_boundary    = 'vacuum',
        
        # BELLHOP: Top interface
        top_interface   = np.column_stack((top_range,top_interface)),          # m
        
        # ALL: Sound speed profiles
        ssp_range       = ssp_range,                                           # m
        ssp_depth       = ssp_depth,                                           # m
        ssp             = ssp,                                                 # m/s

        # OALIB: Sound speed interpolation
        ssp_interp      = 'c-linear',
        
        # ALL: Source freq and depth
        tx_freq         = tx_freq,                                             # Hz
        tx_depth        = tx_depth,                                            # m
        
        # BELLHOP: Source directivity
        tx_beam         = np.column_stack((tx_angle,tx_level)),                # degree, dB
        tx_nbeam        = 0,                                                   # 0 = automatic
        tx_minAngle     = -180,                                                # deg                         
        tx_maxAngle     = 180,                                                 # deg

        # ALL: Bottom interface
        bot_interface   = np.column_stack((bot_range,bot_interface)),          # m
                                                        
        # OALIB: Bottom boundary 
        bot_boundary    = 'acousto-elastic',
        attn_unit       = 'dB/wavelength',
        bot_roughness   = 0.2,                                                 # m (rms)
        bot_density     = 2.75,                                                # g/cm3
        bot_PwaveSpeed  = 6000,                                                # m/s 
        bot_SwaveSpeed  = 3500,                                                # m/s 
        bot_PwaveAttn   = 0.02,                                                # dB/wavelength 
        bot_SwaveAttn   = 0.02,                                                # dB/wavelength 
        )

BELLHOP = pm.BELLHOP(env)
BELLHOP.compute_transmission_loss()

BELLHOP.plot_transmission_loss()
BELLHOP.plot_ssp(vmin=1000, vmax=2000)

env['rx_range'] = 8000
env['rx_depth'] = 250
env = pm.make_env2d(**env)    
env = BELLHOP.set_env(env)

BELLHOP.compute_arrivals()
BELLHOP.compute_impulse_respsonse(fs=24000, nArrival=100)
BELLHOP.compute_eigen_rays()
BELLHOP.compute_rays()

BELLHOP.plot_arrivals(nArrival=50)  
BELLHOP.plot_impulse_response(nArrival=50) 
BELLHOP.plot_eigen_rays(nRay=12)  
BELLHOP.plot_rays(nRay=12)   
BELLHOP.plot_beam()


                    0.000000e+00       -5.915527e+00        8.445256e-38  \
-15.000000   -0.000194+0.000000j -0.000225+0.000000j -0.000231+0.000000j   
-10.667594   -0.000248+0.000000j -0.000282+0.000000j -0.000297+0.000000j   
-6.335188    -0.000342+0.000000j -0.000343+0.000000j -0.000354+0.000000j   
-2.002782    -0.000580+0.000000j -0.000581+0.000000j -0.000582+0.000000j   
 2.329624    -0.000717+0.000000j -0.000718+0.000000j -0.000711+0.000000j   
...                          ...                 ...                 ...   
 3082.670410 -0.000003+0.000000j -0.000002+0.000000j -0.000003+0.000000j   
 3087.002686 -0.000003+0.000000j -0.000002+0.000000j -0.000003+0.000000j   
 3091.335205 -0.000003+0.000000j -0.000002+0.000000j -0.000003+0.000000j   
 3095.667480 -0.000003+0.000000j -0.000002+0.000000j -0.000003+0.000000j   
 3100.000000 -0.000003+0.000000j -0.000002+0.000000j -0.000003+0.000000j   

                   -5.913548e+00        1.197849e-36       -5.911568e+00  \
-15.000000 

InvalidIndexError: (0, slice(None, None, None))